In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader, Dataset
from PIL import Image


In [18]:

# Create output directory if it doesn't exist
output_dir = r"D:\GAN\Newthird"
os.makedirs(output_dir, exist_ok=True)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_dim = 100
img_height, img_width, channels = 64, 64, 3
batch_size = 32
learning_rate = 0.0001
num_epochs = 40

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*channels, [0.5]*channels)
])


In [19]:

# Custom dataset loader for unlabeled images
class UnlabeledImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [os.path.join(root_dir, file) for file in os.listdir(root_dir) if file.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, 0  # Dummy label

# Dataset and DataLoader
dataset = UnlabeledImageDataset(r"D:\Third hand", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [20]:

# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.LeakyReLU(0.2)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

# Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.init_size = img_height // 16
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 512 * self.init_size ** 2))
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            ResidualBlock(64),
            ResidualBlock(64),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, channels, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 512, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


In [21]:

# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity


In [22]:

# Initialize generator and discriminator
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

# Loss function
adversarial_loss = nn.BCELoss()

# Function to save generated images
def save_generated_images(epoch, generator, latent_dim, output_dir):
    noise = torch.randn(16, latent_dim, device=device)
    generated_images = generator(noise).detach().cpu()
    generated_images = (generated_images + 1) / 2  # Rescale images to [0, 1]
    save_image(generated_images, os.path.join(output_dir, f"gan_generated_image_epoch_{epoch}.png"), nrow=4, normalize=True)


In [23]:
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_images = imgs.to(device)

        # Adversarial ground truths
        valid = torch.ones(real_images.size(0), 1, device=device)
        fake = torch.zeros(real_images.size(0), 1, device=device)

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(real_images.size(0), latent_dim, device=device)

        # Generate a batch of images
        gen_images = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_images), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Loss for real images
        real_loss = adversarial_loss(discriminator(real_images), valid)
        # Loss for fake images
        fake_loss = adversarial_loss(discriminator(gen_images.detach()), fake)
        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        # Print losses
        print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

    # Save generated images
    save_generated_images(epoch, generator, latent_dim, output_dir)
    # Save the generator weights
torch.save(generator.state_dict(), os.path.join(output_dir, 'generator_final.pth'))

# Save the discriminator weights
torch.save(discriminator.state_dict(), os.path.join(output_dir, 'discriminator_final.pth'))


[Epoch 0/40] [Batch 0/38] [D loss: 0.7071085572242737] [G loss: 0.6984012126922607]
[Epoch 0/40] [Batch 1/38] [D loss: 0.5542237162590027] [G loss: 0.7601761221885681]
[Epoch 0/40] [Batch 2/38] [D loss: 0.4746251106262207] [G loss: 0.8660228848457336]
[Epoch 0/40] [Batch 3/38] [D loss: 0.4085870087146759] [G loss: 0.9964646100997925]
[Epoch 0/40] [Batch 4/38] [D loss: 0.34300774335861206] [G loss: 1.1165204048156738]
[Epoch 0/40] [Batch 5/38] [D loss: 0.2737886905670166] [G loss: 1.4092862606048584]
[Epoch 0/40] [Batch 6/38] [D loss: 0.32722747325897217] [G loss: 1.3804874420166016]
[Epoch 0/40] [Batch 7/38] [D loss: 0.2520400285720825] [G loss: 1.3890471458435059]
[Epoch 0/40] [Batch 8/38] [D loss: 0.35537856817245483] [G loss: 1.563973069190979]
[Epoch 0/40] [Batch 9/38] [D loss: 0.3563348650932312] [G loss: 1.3734259605407715]
[Epoch 0/40] [Batch 10/38] [D loss: 0.34665393829345703] [G loss: 1.5800652503967285]
[Epoch 0/40] [Batch 11/38] [D loss: 0.41131848096847534] [G loss: 1.4583

In [26]:
import torch
from torchvision.utils import save_image
import os

# Assume the Generator and Discriminator classes have been defined somewhere

# Initialize the models
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Load the weights from files
generator.load_state_dict(torch.load(r"D:\GAN\Newthird\generator_final.pth"))
discriminator.load_state_dict(torch.load(r"D:\GAN\Newthird\discriminator_final.pth"))

# Set to evaluation mode
generator.eval()
discriminator.eval()

# Specify the directory for saving images
output_dir = r"D:\GAN\Newthird"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Generate images
with torch.no_grad():  # Important for inference to disable gradient calculation
    z = torch.randn(4, latent_dim, device=device)  # 16 is an example batch size for generating 16 images
    generated_images = generator(z)
    image_path = os.path.join(output_dir, 'generated_images2.png')
    save_image(generated_images, image_path, nrow=4, normalize=True)

print(f"Generated images saved to '{image_path}'.")

Generated images saved to 'D:\GAN\Newthird\generated_images2.png'.


In [33]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("duongna/ldm-super-resolution")

unet\diffusion_pytorch_model.safetensors not found
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...: 100%|██████████| 3/3 [00:02<00:00,  1.33it/s]


In [34]:
generator.eval()

# Load the super-resolution model from Hugging Face
pipeline = DiffusionPipeline.from_pretrained("duongna/ldm-super-resolution")
pipeline = pipeline.to("cuda" if torch.cuda.is_available() else "cpu")


unet\diffusion_pytorch_model.safetensors not found
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...: 100%|██████████| 3/3 [00:02<00:00,  1.47it/s]
